#import h2o and run

In [2]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_121"; Java(TM) SE Runtime Environment (build 1.8.0_121-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.121-b13, mixed mode)
  Starting server from /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/s0/d7szw5jj2890t7dmcj8gl9w40000gn/T/tmpc2lxms1m
  JVM stdout: /var/folders/s0/d7szw5jj2890t7dmcj8gl9w40000gn/T/tmpc2lxms1m/h2o_zhaomengqi_started_from_python.out
  JVM stderr: /var/folders/s0/d7szw5jj2890t7dmcj8gl9w40000gn/T/tmpc2lxms1m/h2o_zhaomengqi_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster version:,3.14.0.7
H2O cluster version age:,"14 days, 18 hours and 38 minutes"
H2O cluster name:,H2O_from_python_zhaomengqi_6xausm
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


#Convert Pandas data frame into H2o frame

In [3]:
import pandas as pd

df = pd.read_csv('midterm_2016.csv')

df.drop(['parcelid','airconditioningtypeid'],axis=1,inplace=True)
df.drop(['buildingqualitytypeid','finishedfloor1squarefeet','finishedsquarefeet15','finishedsquarefeet50','fips','fireplacecnt','garagecarcnt','garagetotalsqft', 'latitude','longitude','lotsizesquarefeet','poolcnt','poolsizesum','pooltypeid10','pooltypeid2','pooltypeid7','propertylandusetypeid','rawcensustractandblock','regionidcity','regionidcounty','regionidneighborhood','regionidzip','roomcnt','threequarterbathnbr','unitcnt','yardbuildingsqft17','yearbuilt','numberofstories','taxvaluedollarcnt','assessmentyear','landtaxvaluedollarcnt','taxamount','taxdelinquencyyear','censustractandblock'],axis=1,inplace=True)

wine=h2o.H2OFrame(df)

wine.head(5)





Parse progress: |█████████████████████████████████████████████████████████| 100%


logerror,bathroomcnt,bedroomcnt,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,heatingorsystemtypeid,structuretaxvaluedollarcnt
0.0276,2,3,2,1684,1684,2,2,122754
-0.1684,3.5,4,3.5,2263,2263,3,3.92698,346458
-0.004,3,2,3,2217,2217,3,2,61994
0.0218,2,2,2,839,839,2,2,171518
-0.005,2.5,4,2.5,2283,2283,2,3.92698,169574


In [4]:
feature = list(wine.columns)
feature.remove('logerror')
feature

['bathroomcnt',
 'bedroomcnt',
 'calculatedbathnbr',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'fullbathcnt',
 'heatingorsystemtypeid',
 'structuretaxvaluedollarcnt']

In [5]:
wine_split=wine.split_frame(ratios=[0.75])
wine_train=wine_split[0]
wine_test=wine_split[1]

In [6]:
wine_train.shape

(67825, 9)

In [7]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
glm_default=H2OGeneralizedLinearEstimator(family='gaussian',model_id='glm_default')

In [8]:
glm_default.train(x=feature,y='logerror',training_frame=wine_train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [9]:

glm_default

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  glm_default


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 0.026261115373876945
RMSE: 0.16205281661815368
MAE: 0.06837790575463598
RMSLE: NaN
R^2: 0.0016267425290013016
Mean Residual Deviance: 0.026261115373876945
Null degrees of freedom: 67824
Residual degrees of freedom: 67816
Null deviance: 1784.0623603490517
Residual deviance: 1781.1601502332037
AIC: -54361.33013131289
Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2017-11-04 13:16:20,0.000 sec,0,1784.0623587,0.0263039


In [105]:
yhat_test_glm =glm_default.predict(wine_test)
yhat_test_glm

glm prediction progress: |████████████████████████████████████████████████| 100%


predict
0.0134297
0.0131736
0.00502152
0.0167455
0.015629
0.00772776
0.00880944
0.00505806
0.00958128
0.0256804


In [10]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
drf_default=H2ORandomForestEstimator(seed=1234,model_id='drf_default')
drf_default.train(x=feature,y='logerror',training_frame=wine_train)
drf_default

drf Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  drf_default


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.027277447300287215
RMSE: 0.16515885474381087
MAE: 0.07174529551321478
RMSLE: NaN
Mean Residual Deviance: 0.027277447300287215
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2017-11-04 13:16:29,0.168 sec,0.0,nan,nan,nan
,2017-11-04 13:16:29,0.769 sec,1.0,0.1744463,0.0780717,0.0304315
,2017-11-04 13:16:29,1.023 sec,2.0,0.1798410,0.0785559,0.0323428
,2017-11-04 13:16:30,1.268 sec,3.0,0.1787278,0.0777237,0.0319436
,2017-11-04 13:16:30,1.501 sec,4.0,0.1776213,0.0771563,0.0315493
---,---,---,---,---,---,---
,2017-11-04 13:16:32,3.739 sec,20.0,0.1673755,0.0730828,0.0280146
,2017-11-04 13:16:32,3.826 sec,21.0,0.1669961,0.0729777,0.0278877
,2017-11-04 13:16:32,4.038 sec,22.0,0.1669996,0.0729852,0.0278889
,2017-11-04 13:16:33,4.173 sec,23.0,0.1669483,0.0729090,0.0278717



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
structuretaxvaluedollarcnt,5521.6499023,1.0,0.3324703
calculatedfinishedsquarefeet,4180.2177734,0.7570595,0.2516998
finishedsquarefeet12,3835.9299316,0.6947072,0.2309695
bedroomcnt,1136.2436523,0.2057797,0.0684157
heatingorsystemtypeid,580.9459839,0.1052124,0.0349800
fullbathcnt,472.7566223,0.0856187,0.0284657
bathroomcnt,457.4491882,0.0828465,0.0275440
calculatedbathnbr,422.7562256,0.0765634,0.0254551


In [11]:
drf_test_glm =drf_default.predict(wine_test)
drf_test_glm

drf prediction progress: |████████████████████████████████████████████████| 100%


predict
0.0142056
0.00969776
0.0279526
0.00725515
0.0121618
0.027
0.0064791
0.000176644
0.0105517
0.00452557


In [12]:
feature = list(wine.columns)
#feature.remove('logerror')
feature.remove('calculatedfinishedsquarefeet')
feature.remove('calculatedbathnbr')
feature.remove('structuretaxvaluedollarcnt')
feature.remove('bedroomcnt')
feature.remove('bathroomcnt')
feature.remove('fullbathcnt')

feature

['logerror', 'finishedsquarefeet12', 'heatingorsystemtypeid']

In [13]:
wine_split=wine.split_frame(ratios=[0.75])
wine_train=wine_split[0]
wine_test=wine_split[1]

In [14]:
wine_train.shape

(67618, 9)

In [15]:
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
glm_default=H2OGeneralizedLinearEstimator(family='gaussian',model_id='glm_default')

In [16]:
glm_default.train(x=feature,y='logerror',training_frame=wine_train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [17]:
from h2o.estimators.random_forest import H2ORandomForestEstimator
drf_default=H2ORandomForestEstimator(seed=1234,model_id='drf_default')
drf_default.train(x=feature,y='logerror',training_frame=wine_train)
drf_default

drf Model Build progress: |███████████████████████████████████████████████| 100%
Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  drf_default


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 0.02624302485907395
RMSE: 0.1619969902778257
MAE: 0.06841834701650601
RMSLE: NaN
Mean Residual Deviance: 0.02624302485907395
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2017-11-04 13:18:24,0.074 sec,0.0,nan,nan,nan
,2017-11-04 13:18:24,0.194 sec,1.0,0.1653392,0.0690475,0.0273370
,2017-11-04 13:18:24,0.220 sec,2.0,0.1647754,0.0686980,0.0271509
,2017-11-04 13:18:24,0.253 sec,3.0,0.1610458,0.0682296,0.0259357
,2017-11-04 13:18:24,0.283 sec,4.0,0.1603963,0.0684977,0.0257270
---,---,---,---,---,---,---
,2017-11-04 13:18:27,2.508 sec,46.0,0.1619917,0.0684279,0.0262413
,2017-11-04 13:18:27,2.537 sec,47.0,0.1619950,0.0684243,0.0262424
,2017-11-04 13:18:27,2.565 sec,48.0,0.1619913,0.0684187,0.0262412
,2017-11-04 13:18:27,2.654 sec,49.0,0.1620032,0.0684251,0.0262450



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
finishedsquarefeet12,1083.0378418,1.0,0.8939766
heatingorsystemtypeid,128.4455414,0.1185975,0.1060234


In [18]:
drf_test_glm =drf_default.predict(wine_test)
drf_test_glm

drf prediction progress: |████████████████████████████████████████████████| 100%


predict
0.0128764
0.00756895
0.00750768
0.012224
0.0106338
0.0108733
0.0105962
0.0112595
0.0094614
0.0127927


In [19]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,4 mins 26 secs
H2O cluster version:,3.14.0.7
H2O cluster version age:,"14 days, 18 hours and 43 minutes"
H2O cluster name:,H2O_from_python_zhaomengqi_6xausm
H2O cluster total nodes:,1
H2O cluster free memory:,1.637 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [22]:
import pandas as pd

df = pd.read_csv('midterm_2017.csv')

#df.drop(['parcelid'],axis=1,inplace=True)
#df.drop(['taxdelinquencyflag','propertyzoningdesc','propertycountylandusecode','hashottuborspa','transactiondate','transaction_month','fireplaceflag','basementsqft','buildingclasstypeid','decktypeid','finishedsquarefeet13','yardbuildingsqft26','typeconstructiontypeid','storytypeid','finishedsquarefeet6'],axis=1,inplace=True)

wine=h2o.H2OFrame(df)

wine.head(5)


Parse progress: |█████████████████████████████████████████████████████████| 100%


parcelid,logerror,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet15,finishedsquarefeet50,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid2,pooltypeid7,propertylandusetypeid,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,threequarterbathnbr,unitcnt,yardbuildingsqft17,yearbuilt,numberofstories,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock
1.42975e+07,0.0255949,1.81201,3.5,4,6.53378,3.5,1366.17,3100,3100,2354.49,1381.39,6059,1.19122,3,2,633,3.92175,3.36349e+07,-1.17869e+08,4506,1,517.931,1,1,261,6.05906e+07,53571,1286,187725,96978,0,1,1.11031,305.461,1998,1.43429,485713,1.02328e+06,2016,537569,11013.7,14.0883,6.05906e+13
1.70529e+07,0.0556191,1.81201,1,2,6.53378,1,1465,1465,1465,2354.49,1465,6111,1,1,1,0,3.92175,3.44493e+07,-1.19282e+08,12647,1,517.931,1,1,261,6.111e+07,13091,2061,187725,97099,5,1.0096,1.11031,305.461,1967,1,88000,464000,2016,376000,5672.48,14.0883,6.111e+13
1.41862e+07,0.00538285,1.81201,2,3,6.53378,2,1366.17,1243,1243,2354.49,1381.39,6059,1.19122,2,2,440,3.92175,3.38862e+07,-1.17823e+08,8432,1,517.931,1,1,261,6.05902e+07,21412,1286,187725,97078,6,1.0096,1.11031,305.461,1962,1,85289,564778,2016,479489,6488.3,14.0883,6.05902e+13
1.21779e+07,-0.10341,1.81201,3,4,8,3,1366.17,2376,2376,2354.49,1381.39,6037,1.19122,3,1.81528,350.047,2,3.42452e+07,-1.18241e+08,13038,1,517.931,1,1,261,6.0373e+07,396551,3101,187725,96330,0,1.0096,1,305.461,1970,1.43429,108918,145143,2016,36225,1777.51,14.0883,6.0373e+13
1.08872e+07,0.00694036,1,3,3,8,3,1366.17,1312,1312,2354.49,1381.39,6037,1.19122,3,1.81528,350.047,2,3.41851e+07,-1.18415e+08,278581,1,517.931,1,1,266,6.03712e+07,12447,3101,268548,96451,0,1.0096,1,305.461,1964,1.43429,73681,119407,2016,45726,1533.89,14.0883,6.03712e+13
